In [26]:
from keras import backend as K
K.clear_session()

In [27]:
import keras
from keras.applications.vgg16 import VGG16

from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.models import Model
from keras.models import Sequential


In [28]:
from __future__ import print_function
import time
import numpy as np
import pandas as pd
from PIL import Image
import glob
import matplotlib.pyplot as plt
import os

In [29]:
import random
from shutil import copyfile

# Copyright 2014-2017 Bert Carremans
# Author: Bert Carremans <bertcarremans.be>
#
# License: BSD 3 clause


def img_train_test_split(img_source_dir, train_size):   
    if not (isinstance(img_source_dir, str)):
        raise AttributeError('img_source_dir must be a string')
        
    if not (isinstance(train_size, float)):
        raise AttributeError('train_size must be a float')
        
    # Set up empty folder structure if not exists
    if not os.path.exists('data'):
        os.makedirs('data')
    else:
        if not os.path.exists('data/train'):
            os.makedirs('data/train')
        if not os.path.exists('data/validation'):
            os.makedirs('data/validation')
    
    # Get the subdirectories in the main image folder
    
    subdirs = [subdir for subdir in os.listdir(img_source_dir) if os.path.isdir(os.path.join(img_source_dir, subdir))]
    
    for subdir in subdirs:
        subdir_fullpath = os.path.join(img_source_dir, subdir)
        if len(os.listdir(subdir_fullpath)) == 0:
            print(subdir_fullpath + ' is empty')
            break

        train_subdir = os.path.join('data/train', subdir)
        validation_subdir = os.path.join('data/validation', subdir)

        # Create subdirectories in train and validation folders
        if not os.path.exists(train_subdir):
            os.makedirs(train_subdir)

        if not os.path.exists(validation_subdir):
            os.makedirs(validation_subdir)

        train_counter = 0
        validation_counter = 0

        # Randomly assign an image to train or validation folder
        for filename in os.listdir(subdir_fullpath):
            if filename.endswith(".jpg") or filename.endswith(".png"): 
                fileparts = filename.split('.')

                if random.uniform(0, 1) <= train_size:
                    copyfile(os.path.join(subdir_fullpath, filename), os.path.join(train_subdir, str(train_counter) + '.' + fileparts[1]))
                    train_counter += 1
                else:
                    copyfile(os.path.join(subdir_fullpath, filename), os.path.join(validation_subdir, str(validation_counter) + '.' + fileparts[1]))
                    validation_counter += 1
                    
        print('Copied ' + str(train_counter) + ' images to data/train/' + subdir)
        print('Copied ' + str(validation_counter) + ' images to data/validation/' + subdir)

In [30]:
img_path = "/home/sogang03/school/ClimateChange_tsne/data/128px_image_square_final"
images = glob.glob(os.path.join(img_path,"*/*.jpg" ))


img_train_test_split(img_path, 0.7)

Copied 88 images to data/train/korean_rice
Copied 43 images to data/validation/korean_rice
Copied 29 images to data/train/disposable_diaper
Copied 6 images to data/validation/disposable_diaper
Copied 170 images to data/train/coffee
Copied 63 images to data/validation/coffee
Copied 42 images to data/train/china_wheat
Copied 21 images to data/validation/china_wheat
Copied 52 images to data/train/quiver_tree
Copied 22 images to data/validation/quiver_tree
Copied 204 images to data/train/aluminum_can
Copied 84 images to data/validation/aluminum_can
Copied 57 images to data/train/styrofoam
Copied 28 images to data/validation/styrofoam
Copied 6 images to data/train/spruce_budworm
Copied 1 images to data/validation/spruce_budworm
Copied 4 images to data/train/styrofoam_buoy
Copied 1 images to data/validation/styrofoam_buoy
Copied 98 images to data/train/newt
Copied 44 images to data/validation/newt
Copied 76 images to data/train/carpenter_bee
Copied 30 images to data/validation/carpenter_bee


In [31]:
import os
import glob

a = glob.glob('/home/sogang03/school/ClimateChange_tsne/data/train/*')
class_names = []
for i in range(len(a)):
    p = a[i][52:]
    class_names.append(p)
    
print(class_names)

['korean_rice', 'disposable_diaper', 'coffee', 'china_wheat', 'quiver_tree', 'aluminum_can', 'styrofoam', 'spruce_budworm', 'styrofoam_buoy', 'newt', 'carpenter_bee', 'long_distance_migrant_bird', 'plastic_bag', 'ink_cartridge', 'barley', 'spring_peeper', 'light_bulb', 'mason_bee', 'tiger', 'frog', 'adélie_penguin', 'electronic_waste', 'tin_can', 'polar_bear', 'ivory_gull', 'coral', 'plastic_bottle', 'gecko', 'jack_pine_budworm', 'fishing_line', 'plastic_beverage_holder', 'crocodilian', 'lizard', 'glass_bottle', 'bumblebee', 'banana', 'phytoplankton', 'snake', 'forest_bird', 'shark', 'lion', 'corn', 'snowy_plover', 'toad', 'cacao']


In [32]:
#preprocess image 

from keras.preprocessing.image import ImageDataGenerator

train_data_dir = './data/train'
validation_data_dir = './data/validation'
input_shape=(224,224,3)

trdata = ImageDataGenerator( 
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
traindata = trdata.flow_from_directory(directory= train_data_dir,classes=class_names, color_mode='rgb', class_mode='categorical', target_size=(224,224), batch_size=16)
vldata = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
valdata = vldata.flow_from_directory(directory= validation_data_dir,classes=class_names,  color_mode='rgb', class_mode='categorical', target_size=(224,224), batch_size=16)

Found 5149 images belonging to 45 classes.
Found 2365 images belonging to 45 classes.


In [33]:
model = VGG16(include_top=False,  input_shape=(224, 224, 3)) #pretrained weight : image-net
for layer in model.layers:
    layer.trainable = False
# add new classifier layers
flat1 = Flatten()(model.outputs)
class1 = Dense(4096, activation='relu')(flat1)

# define new model
model = Model(inputs=model.inputs, outputs=output)
# summarize
model.summary()



ValueError: Graph disconnected: cannot obtain value for tensor Tensor("input_6:0", shape=(?, 224, 224, 3), dtype=float32) at layer "input_6". The following previous layers were accessed without issue: []

In [20]:
# model = Sequential()
# model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
#                  activation='relu',
#                  input_shape=(224,224,3)))
# model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
# model.add(Conv2D(64, (5, 5), activation='relu'))
# model.add(MaxPool2D(pool_size=(2, 2)))
# model.add(Flatten())
# model.add(Dense(100, activation='relu'))
# model.add(Dense(45, activation='softmax'))

# model.summary()

In [21]:
print("traindata", len(traindata))
print("valdata", len(valdata))

traindata 322
valdata 148


In [22]:
from keras.optimizers import Adam

opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [23]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("/tmp/weights.hdf5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')

hist = model.fit_generator(steps_per_epoch=200,generator=traindata, validation_data= valdata, validation_steps=10,epochs=100,callbacks=[checkpoint,early])

Epoch 1/100


InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: Matrix size-incompatible: In[0]: [1,401408], In[1]: [25088,1024]
	 [[{{node dense_5/MatMul}}]]
	 [[Mean_2/_1335]]
  (1) Invalid argument: Matrix size-incompatible: In[0]: [1,401408], In[1]: [25088,1024]
	 [[{{node dense_5/MatMul}}]]
0 successful operations.
0 derived errors ignored.

In [14]:
# Save the model
model.save('climateChange_1.h5')